In [1]:
import csv
import random
import os


In [8]:
# Function to assign CPU consumption based on thirds
def set_cpu_thresholds(remaining_entries, current_index, total_entries):
    global initial_cpu
    global applied_cpu

    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_cpu = round(random.uniform(0.2,1.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.8, 0.99), 2)  # Quota of 80% to 99%
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_cpu = round(random.uniform(2.0, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.1, 0.99), 2)  # Quota of 10% to 99%
    else:
        # Third third equal to second third
        initial_cpu = round(random.uniform(2, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.1, 0.99), 2)  # Quota of 10% to 99%
# Function to assign memory consumption based on thirds
def set_memory_thresholds(remaining_entries, current_index, total_entries):
    global initial_avg_memory_consumption
    global applied_memory_consumption
    
    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_avg_memory_consumption = round(random.uniform(80, 199), 2)
        applied_memory_consumption = round(random.uniform(0.8, 0.99), 2)
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_avg_memory_consumption = round(random.uniform(200, 799), 2)
        applied_memory_consumption = round(random.uniform(0.5, 0.99), 2) # Quota of 50% to 99%
    else:
        # Third third
        initial_avg_memory_consumption = round(random.uniform(800, 2000), 2)
        applied_memory_consumption = round(random.uniform(0.1, 0.99), 2)    # Quota of 10% to 99%


# Function to assign CPU consumption based on thirds
def set_cpu_thresholds_latency(remaining_entries, current_index, total_entries):
    global initial_cpu
    global applied_cpu

    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_cpu = round(random.uniform(0.2,1.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.8, 0.99), 2)  # Quota of 80% to 99%
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_cpu = round(random.uniform(2.0, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.1, 0.99), 2)  # Quota of 10% to 99%
    else:
        # Third third equal to second third
        initial_cpu = round(random.uniform(2, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.1, 0.99), 2)  # Quota of 10% to 99%
# Function to assign memory consumption based on thirds
def set_memory_thresholds_latency(remaining_entries, current_index, total_entries):
    global initial_avg_memory_consumption
    global applied_memory_consumption
    
    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_avg_memory_consumption = round(random.uniform(80, 199), 2)
        applied_memory_consumption = round(random.uniform(0.1, 1.99), 2)
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_avg_memory_consumption = round(random.uniform(200, 799), 2)
        applied_memory_consumption = round(random.uniform(0.2, 1.99), 2) # Quota of 50% to 99%
    else:
        # Third third
        initial_avg_memory_consumption = round(random.uniform(800, 2000), 2)
        applied_memory_consumption = round(random.uniform(0.1, 1.99), 2)    # Quota of 10% to 99%

     # No changes in memory for this case
def apply_scaling_memory(ranking, scale_up):
    """
    Apply scaling logic to determine whether to scale up or scale down memory consumption.

    Args:
        initial_avg_memory_consumption (float): The initial average memory consumption.
        replicas (int): The number of replicas.

    Returns:
        tuple: A tuple containing:
            - applied_memory_consumption (float): The adjusted memory consumption.
            - label (int): The label indicating scaling direction (2 for scale up, 3 for scale down).
            - skip (bool): Whether to skip this entry (True if the entry should be skipped).
    """
    global initial_avg_memory_consumption
    global applied_memory_consumption
    global label
    # Compute the minimum required change (90% of initial/replicas)
    min_required_change = 0.9 * (initial_avg_memory_consumption / replicas)
    min_percentage= min_required_change/initial_avg_memory_consumption


    # Check if scaling down is invalid for a single replica
    if replicas == 1 and not scale_up:
        scale_up = True

    # Apply scaling logic
    if scale_up:  # Scale up case
        applied_memory_consumption = round(initial_avg_memory_consumption +  (initial_avg_memory_consumption* random.uniform(min_percentage, min_percentage * (replicas/2))), 2)
        label = 2
    else:  # Scale down case
        if ranking ==1 :
            applied_memory_consumption =  round(initial_avg_memory_consumption-random.uniform(min_required_change, min_required_change*(replicas/2)), 2)
        elif ranking ==2:
             applied_memory_consumption = round( initial_avg_memory_consumption-random.uniform(min_required_change, min_required_change*(replicas/2)), 2)
        elif ranking ==3:
              applied_memory_consumption =  round(initial_avg_memory_consumption-random.uniform(min_required_change, min_required_change*(replicas/2)), 2)

        label = 3



def apply_scaling_cpu(ranking, scale_up):
    """
    Apply scaling logic to determine whether to scale up or scale down memory consumption.

    Args:
        initial_avg_memory_consumption (float): The initial average memory consumption.
        replicas (int): The number of replicas.

    Returns:
        tuple: A tuple containing:
            - applied_memory_consumption (float): The adjusted memory consumption.
            - label (int): The label indicating scaling direction (2 for scale up, 3 for scale down).
            - skip (bool): Whether to skip this entry (True if the entry should be skipped).
    """
    global initial_cpu
    global applied_cpu
    global label
    # Compute the minimum required change (90% of initial/replicas)
    min_required_change = 0.9 * (initial_cpu / replicas)
    min_percentage= min_required_change/initial_cpu
    # Decide scaling direction
    

    # Check if scaling down is invalid for a single replica
    if replicas == 1 and not scale_up:
        scale_up = True

    # Apply scaling logic
    if scale_up:  # Scale up case
        applied_cpu = round(initial_cpu +  (initial_cpu*random.uniform(min_percentage, min_percentage * (replicas/2))), 2)
        label = 2
    else:  # Scale down case
        if ranking ==1 :
            applied_cpu = round(initial_cpu - (initial_cpu*random.uniform(min_percentage, min_percentage * (replicas/2))), 2)
        elif ranking ==2:
             applied_cpu = round(initial_cpu - (initial_cpu*random.uniform(min_percentage, min_percentage * (replicas/2))), 2)
        elif ranking ==3:
              applied_cpu = round(initial_cpu -  (initial_cpu*random.uniform(min_percentage, min_percentage * (replicas/2))), 2)
        label = 3        
# Function to assign CPU consumption based on thirds
def set_cpu_thresholds_scaling(remaining_entries, current_index, total_entries, scale_up):
    global initial_cpu
    global applied_cpu

    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_cpu = round(random.uniform(1,2), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.8, 0.99), 2)  # Quota of 80% to 99%
        apply_scaling_cpu(1, scale_up)
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_cpu = round(random.uniform(2.0, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.1, 0.99), 2)  # Quota of 10% to 99%
        apply_scaling_cpu(2, scale_up)
    else:
        # Third third equal to second third
        initial_cpu = round(random.uniform(4, 10), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.1, 0.99), 2)  # Quota of 10% to 99%
        apply_scaling_cpu(3, scale_up)
# Function to assign memory consumption based on thirds
def set_memory_thresholds_scaling(remaining_entries, current_index, total_entries, scale_up):
    global initial_avg_memory_consumption
    global applied_memory_consumption
    global replicas
    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_avg_memory_consumption = round(random.uniform(200, 600), 2)
        #applied_memory_consumption = round(random.uniform(0.8, 0.99), 2)
        apply_scaling_memory(1, scale_up)
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_avg_memory_consumption = round(random.uniform(600, 1000), 2) 
        #applied_memory_consumption = round(random.uniform(0.5, 0.99), 2) # Quota of 50% to 99%
        apply_scaling_memory(2, scale_up)
    else:
        # Third third
        initial_avg_memory_consumption = round(random.uniform(1000, 2000) , 2)
        #applied_memory_consumption = round(random.uniform(0.1, 0.99), 2)    # Quota of 10% to 99%
        apply_scaling_memory(3, scale_up)

# Function to assign CPU consumption based on thirds
def set_cpu_thresholds_migration(remaining_entries, current_index, total_entries):
    global initial_cpu
    global applied_cpu

    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_cpu = round(random.uniform(0.2,1.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.01, 0.15), 2) # Quota of 80% to 99%
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_cpu = round(random.uniform(2.0, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.01, 0.15), 2) # Quota of 10% to 99%
    else:
        # Third third equal to second third
        initial_cpu = round(random.uniform(2, 8.0), 2)  # Example CPU range
        applied_cpu = round(random.uniform(0.01, 0.15), 2)  # Quota of 10% to 99%
# Function to assign memory consumption based on thirds
def set_memory_thresholds_migration(remaining_entries, current_index, total_entries):
    global initial_avg_memory_consumption
    global applied_memory_consumption
    
    third_size = remaining_entries // 3  # Divide remaining entries into three parts
    threshold_index = total_entries - remaining_entries  # Starting index of remaining entries

    if current_index < threshold_index + third_size:
        # First third
        initial_avg_memory_consumption = round(random.uniform(80, 199), 2)
        applied_memory_consumption = round(random.uniform(0.01, 0.15), 2)
    elif current_index < threshold_index + 2 * third_size:
        # Second third
        initial_avg_memory_consumption = round(random.uniform(200, 799), 2)
        applied_memory_consumption = round(random.uniform(0.01, 0.15), 2) # Quota of 50% to 99%
    else:
        # Third third
        initial_avg_memory_consumption = round(random.uniform(800, 1200), 2)
        applied_memory_consumption = round(random.uniform(0.01, 0.15), 2)    # Quota of 10% to 99%


In [10]:
# 0 : optimize and should apply if there are only small differences between initial and applied memory (around 29%). Valid for any number of replicas 
import os
import csv
import random

# Number of entries to generate
num_entries = 2000 # Adjust as needed
deployment_ids = random.sample(range(10000000, 99999999), num_entries)

# Generate the data
data = []
for i in range(num_entries):
    deployment_id = deployment_ids[i]
    replicas = random.randint(1, 6)  # Any number of replicas
    if i <= num_entries * 0.33:  
        remaining_entries = int(num_entries * 0.33)-1
        set_memory_thresholds(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        
    if i > num_entries * 0.33 and   i < num_entries * 0.66: 
        remaining_entries = int(num_entries * 0.66)- int(num_entries * 0.33)
        set_memory_thresholds(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        applied_cpu = 1 
    if i >= num_entries * 0.66: # generate cases where no changes in memory
        remaining_entries = num_entries - int(num_entries * 0.66)
        set_memory_thresholds(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        applied_memory_consumption = 1
 
    initial_avg_latency = round(random.uniform(1, 60), 2)
    latency_variation = random.uniform(-0.05, 0.05)  # 5% range latency tolerance
    applied_latency = round(initial_avg_latency * (1 + latency_variation), 2)

    data.append({
        'deployment_id': deployment_id,
        'replicas': 1,
        'initial_cpu': initial_cpu,
        'applied_cpu': round(applied_cpu*initial_cpu,2),  # Percentage format
        'initial_avg_memory_consumption': initial_avg_memory_consumption,
        'applied_memory_consumption': round(applied_memory_consumption*initial_avg_memory_consumption,2),  # Percentage format
        'initial_avg_latency': initial_avg_latency,
        'applied_latency': applied_latency,
        'label': 0  # Optimization case
    })

# Ensure directory exists
os.makedirs("./datasets", exist_ok=True)
file_path = "./datasets/df1_0_optimize.csv"

# Write to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file saved at {file_path}")


CSV file saved at ./datasets/df1_0_optimize.csv


In [11]:
# 1 : If latency is high or 0 > return None
import os
import csv
import random

# Number of entries to generate
num_entries = 3000 # Adjust as needed
deployment_ids = random.sample(range(10000000, 99999999), num_entries)

# Generate the data
data = []
for i in range(num_entries):
    deployment_id = deployment_ids[i]
    replicas = random.randint(1, 6)  # Any number of replicas
    if i <= num_entries * 0.33:  
        remaining_entries = int(num_entries * 0.33)-1
        set_memory_thresholds_latency(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        initial_avg_latency = round(random.uniform(1, 60), 2)
        latency_variation = random.uniform(0.08, 1) 
        applied_latency = round(initial_avg_latency * (1 + latency_variation), 2)
    if i > num_entries * 0.33 and   i < num_entries * 0.66: 
        remaining_entries = int(num_entries * 0.66)- int(num_entries * 0.33)
        set_memory_thresholds_latency(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        applied_cpu = 1 
        initial_avg_latency = round(random.uniform(1, 60), 2)
        latency_variation = random.uniform(0.08, 1) 
        applied_latency = round(initial_avg_latency * (1 + latency_variation), 2)
    if i >= num_entries * 0.66 and   i < num_entries * 0.90: # generate cases where no changes in memory
        remaining_entries = num_entries - int(num_entries * 0.66)
        set_memory_thresholds_latency(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        applied_memory_consumption = 1
        initial_avg_latency = round(random.uniform(1, 60), 2)
        latency_variation = random.uniform(0.08, 1) 
        applied_latency = round(initial_avg_latency * (1 + latency_variation), 2)
    if i > num_entries * 0.90: 
        remaining_entries = int(num_entries * 0.66)- int(num_entries * 0.33)
        set_memory_thresholds_latency(remaining_entries, i, num_entries)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        initial_avg_latency = round(random.uniform(1, 60), 2)
        applied_latency = -1
        

    data.append({
        'deployment_id': deployment_id,
        'replicas': replicas,
        'initial_cpu': initial_cpu,
        'applied_cpu': round(applied_cpu*initial_cpu,2),  # Percentage format
        'initial_avg_memory_consumption': initial_avg_memory_consumption,
        'applied_memory_consumption': round(applied_memory_consumption*initial_avg_memory_consumption,2),  # Percentage format
        'initial_avg_latency': initial_avg_latency,
        'applied_latency': applied_latency,
        'label': 1  # Optimization case
    })

# Ensure directory exists
os.makedirs("./datasets", exist_ok=True)
file_path = "./datasets/df2_1_latency.csv"

# Write to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file saved at {file_path}")


CSV file saved at ./datasets/df2_1_latency.csv


In [14]:
# Number of entries to generate
num_entries = 5000  # Adjust as needed
deployment_ids = random.sample(range(10000000, 99999999), num_entries)

# Generate the data
data = []
for i in range(num_entries):
    deployment_id = deployment_ids[i]
    replicas = random.randint(1, 6)  # Any number of replicas
    scale_up = random.choice([True, False])
    if i <= num_entries * 0.33:  # First third
        remaining_entries = int(num_entries * 0.33)
        set_memory_thresholds_scaling(remaining_entries, i, num_entries, scale_up)
        set_cpu_thresholds_scaling(remaining_entries, i, num_entries, scale_up)
    elif i > num_entries * 0.33 and i <= num_entries * 0.66:  # Second third
        remaining_entries = int(num_entries * 0.66) - int(num_entries * 0.33)
        set_memory_thresholds_scaling(remaining_entries, i, num_entries, scale_up)
        set_cpu_thresholds(remaining_entries, i, num_entries)
        applied_cpu = 1  # No changes in CPU
    else:  # Third third
        remaining_entries = num_entries - int(num_entries * 0.66)
        set_memory_thresholds(remaining_entries, i, num_entries)
        set_cpu_thresholds_scaling(remaining_entries, i, num_entries, scale_up)
        applied_memory_consumption = 1  # No changes in memory

    # Latency logic
    initial_avg_latency = round(random.uniform(1, 60), 2)
    latency_variation = random.uniform(-0.05, 0.05)  # 5% range latency tolerance
    applied_latency = round(initial_avg_latency * (1 + latency_variation), 2)

    # Append the data
    data.append({
        'deployment_id': deployment_id,
        'replicas': replicas,
        'initial_cpu': initial_cpu,
        'applied_cpu': applied_cpu,  # Percentage format
        'initial_avg_memory_consumption': initial_avg_memory_consumption,
        'applied_memory_consumption': applied_memory_consumption,  # Percentage format
        'initial_avg_latency': initial_avg_latency,
        'applied_latency': applied_latency,
        'label': label  # Scaling
    })

# Ensure directory exists
os.makedirs("./datasets", exist_ok=True)
file_path = "./datasets/df3_23_scaling.csv"

# Write to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file saved at {file_path}")

CSV file saved at ./datasets/df3_23_scaling.csv


In [13]:
# 0 : optimize and should apply if there are only small differences between initial and applied memory (around 29%). Valid for any number of replicas 
import os
import csv
import random

# Number of entries to generate
num_entries = 2000 # Adjust as needed
deployment_ids = random.sample(range(10000000, 99999999), num_entries)

# Generate the data
data = []
for i in range(num_entries):
    deployment_id = deployment_ids[i]
    replicas = 1  # Any number of replicas
    if i <= num_entries * 0.33:    # Changes in both
        remaining_entries = int(num_entries * 0.33)-1
        set_memory_thresholds_migration(remaining_entries, i, num_entries)
        set_cpu_thresholds_migration(remaining_entries, i, num_entries)
        
    if i > num_entries * 0.33 and   i < num_entries * 0.66:         #NO cpu changes
        remaining_entries = int(num_entries * 0.66)- int(num_entries * 0.33)
        set_memory_thresholds_migration(remaining_entries, i, num_entries)
        set_cpu_thresholds_migration(remaining_entries, i, num_entries)
        applied_cpu = 1 
    if i >= num_entries * 0.66: # No memory changes
        remaining_entries = num_entries - int(num_entries * 0.66)
        set_memory_thresholds_migration(remaining_entries, i, num_entries)
        set_cpu_thresholds_migration(remaining_entries, i, num_entries)
        applied_memory_consumption = 1
 
    initial_avg_latency = round(random.uniform(1, 60), 2)
    latency_variation = random.uniform(-0.05, 0.05)  # 5% range latency tolerance
    applied_latency = round(initial_avg_latency * (1 + latency_variation), 2)

    data.append({
        'deployment_id': deployment_id,
        'replicas': 1,
        'initial_cpu': initial_cpu,
        'applied_cpu': round(applied_cpu*initial_cpu,2),  # Percentage format
        'initial_avg_memory_consumption': initial_avg_memory_consumption,
        'applied_memory_consumption': round(applied_memory_consumption*initial_avg_memory_consumption,2),  # Percentage format
        'initial_avg_latency': initial_avg_latency,
        'applied_latency': applied_latency,
        'label': 1  # Optimization case
    })

# Ensure directory exists
os.makedirs("./datasets", exist_ok=True)
file_path = "./datasets/df4_1_migrate.csv"

# Write to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file saved at {file_path}")


CSV file saved at ./datasets/df4_1_migrate.csv
